In [59]:
import serial as ser
import numpy as np
from math import pi, cos, sin, radians
import time

### Forward Kinematic (Vorherige Aufgabe)

In [15]:
def forward_kinematic_braccio(joint_angles):
    ''' param: joints: current position of all the joints of the braccio '''
    ''' return: current position'''

    result = np.eye(4)
    
    ## initial translation
    result = result @ translation([281, 0, 3])
    
    ## Joint 0 (Base b)
    result = result @ rotate_z(90 - joint_angles[0]) @ translation([0, 0, 71]) 
    
    ## Joint 1 (Shoulder s)
    result = result @ rotate_y(90 - joint_angles[1]) @ translation([0, 0, 126]) 
    
    ## Joint 2 (Elbow e)
    result = result @ rotate_y(90 - joint_angles[2]) @ translation([0, 0, 124]) 
    
    ## Joint 3 (Wrist tilt v)
    result = result @ rotate_y(90 - joint_angles[3]) @ translation([0, 0, 60])

    ## Joint 4 (Wrist rotate w)
    result = result @ rotate_z(90 - joint_angles[4]) @ translation([0, 0, 134]) 
    
    return result

def translation(trans):
    return np.array([
        [1,0,0,trans[0]],
        [0,1,0,trans[1]],
        [0,0,1,trans[2]],
        [0,0,0,1]
    ])

def rotate_z(theta):
    rad = radians(theta)
    return np.array([
        [cos(rad), -sin(rad), 0, 0],
        [sin(rad), cos(rad), 0, 0],
        [0,0,1,0],
        [0,0,0,1]
    ])

def rotate_y(theta):
    rad = radians(theta)
    return np.array([
        [cos(rad),0, sin(rad), 0],
        [0,1,0,0],
        [-sin(rad),0, cos(rad), 0],
        [0,0,0,1]
    ])

def forward_kinematic_braccio_fixed(joint_angles):
    b = joint_angles[0]
    s = 110
    e = joint_angles[1]
    v = joint_angles[2]
    w = 90
    g = 73
    joint_angles_with_fixed = [b, s, e, v, w, g]
    return forward_kinematic_braccio(joint_angles_with_fixed)

In [39]:
result = forward_kinematic_braccio_fixed([60, 160, 160])[0:3,3]
print(result)

[  78.82944746 -116.72322293   10.10090179]


### Rückwärts Kinematik

In [17]:
def F(x_soll, joint_angles):
    transformation_matrix = forward_kinematic_braccio_fixed(joint_angles)
    x_ist = transformation_matrix[0:3,3]
    return x_ist - x_soll

def dF_theta1(x_soll, joint_angles):
    h = 1E-4
    new_joint_angles = [joint_angles[0]+ h, joint_angles[1], joint_angles[2]]
    return (F(x_soll, new_joint_angles) - F(x_soll, joint_angles)) / h

def dF_theta2(x_soll, joint_angles):
    h = 1E-4
    new_joint_angles = [joint_angles[0], joint_angles[1] + h, joint_angles[2]]
    return (F(x_soll, new_joint_angles) - F(x_soll, joint_angles)) / h

def dF_theta3(x_soll, joint_angles):
    h = 1E-4
    new_joint_angles = [joint_angles[0], joint_angles[1], joint_angles[2] + h]
    return (F(x_soll, new_joint_angles) - F(x_soll, joint_angles)) / h

In [18]:
def jaccobi(x_soll, joint_angles):
    return np.stack((
        dF_theta1(x_soll, joint_angles),
        dF_theta2(x_soll, joint_angles),
        dF_theta3(x_soll, joint_angles)
    ), axis=-1)

def inv_jaccobi(x_soll, joint_angles):
    return np.linalg.pinv(jaccobi(x_soll, joint_angles))

In [19]:
def backward_kinematic_braccio(x_soll, theta_0, max_iter=1000, error=1, alpha=1):
    
    theta_i = theta_0
    
    for i in range(max_iter):
        transformation_matrix = forward_kinematic_braccio_fixed(theta_i)
        x_ist = transformation_matrix[0:3,3]
        diff = np.linalg.norm(x_soll - x_ist)
                
        if diff < error:
            return theta_i
        
        j_inv = inv_jaccobi(x_soll, theta_i)
        theta_i = theta_i - alpha * j_inv.dot(F(x_soll, theta_i))
        
    raise Exception("Sorry, no solution found") 

In [63]:
def linear_backward(x_soll, theta_0, splits=20):
    joints = []
    coordinates = forward_kinematic_braccio_fixed([60, 160, 160])[0:3,3]
    correction = (x_soll - coordinates)/splits
    theta_i = theta_0
    for i in range(splits):
        coordinates += correction
        theta_i = backward_kinematic_braccio(coordinates, theta_i)
        joints.append(theta_i)
    return joints

In [64]:
x_goal = np.array([0, 100, 100])
theta_0 = np.array([60, 160, 160])
joints = linear_backward(x_goal, theta_0)
for joint in joints:
    print(joint)

[ 62.78693836 157.58591187 163.04413909]
[ 65.64214134 155.37232722 165.52570345]
[ 68.51477416 153.26637871 167.67209468]
[ 71.39508918 151.26462093 169.49266769]
[ 74.26846021 149.36601482 170.99476125]
[ 77.12046095 147.57313085 172.18297305]
[ 79.93731657 145.89169493 173.05961073]
[ 82.70627654 144.33002898 173.62530012]
[ 85.41592473 142.89849156 173.87943179]
[ 88.05640083 141.60892812 173.82046431]
[ 90.61952767 140.474158   173.44608906]
[ 93.09884852 139.50753485 172.75325147]
[ 95.48958594 138.72262106 171.73801742]
[ 97.78853835 138.13301617 170.39526682]
[ 99.99393227 137.75237714 168.71818678]
[102.10524768 137.59466955 166.69751856]
[104.12303163 137.67469898 164.32047999]
[106.04871245 138.00899951 161.56922569]
[107.88442338 138.61721673 158.41859787]
[109.6328417  139.52424679 154.83271058]


### Input to Braccio

In [69]:
s = ser.Serial ('COM9', baudrate = 9600, parity = 'N', bytesize = 8, stopbits = 1, timeout = None)

In [52]:
def reset():
    s.write (b'90 b 90 s 90 e 90 v 90 w 60 g\r\n')

In [78]:
def go_to_joint(joint):
    value = f'{int(joint[0])} b 110 s {int(joint[1])} e {int(joint[2])} v 90 w 73 g\r\n'.encode('utf-8')
    s.write (value)

In [82]:
reset()

SerialTimeoutException: Write timeout

In [79]:
go_to_joint(joints[0])

In [80]:
for joint in joints:
    time.sleep(2)
    go_to_joint(joint)

KeyboardInterrupt: 

In [77]:
print(joints[0])

[ 62.78693836 157.58591187 163.04413909]


## Port schliessen

In [68]:
s.close()